In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from collections import defaultdict
from datetime import date
import json

## Scraping Spanish Newspaper: El Pais

This piece of code scrapes newspaper articles from El Pais (Spanish newspaper).
It scrapes the newspaper articles and stores into a list.
Then, this list is saved in a json file. 

In [2]:
url = 'https://elpais.com/america/'
f = urlopen(url)
html_elpais = BeautifulSoup(f, 'lxml')
f.close()

In [3]:
def scraping_el_pais(url, max_num_articles = 1000):
    '''
    This function scrapes the Spanish newspaper El Pais and returns a dictionary which contains information about the scraped articles. 
    ----------------------------------------------------
    Arguments: url (str) - url of the newspaper
               max_num_articles - maximum number of articles to scrape. set to 1000 by default.
    Returns:   el_pais_articles (list) - contains dictionaries of articles: each article has a title, a source, genre, content and readability level. 
    '''
    
    f = urlopen(url)
    html_elpais = BeautifulSoup(f, 'lxml')
    count = 0
    el_pais_articles = []
    
    for node in html_elpais.find_all('a'):
        if node.attrs['href'].startswith('/') and node.attrs['href'].endswith('.html'):
            address = node.attrs['href'].split('/')
            genre = address[1]
            child_url = 'https://elpais.com' + node.attrs['href']
            f_child = urlopen(child_url)
            html_child = BeautifulSoup(f_child, 'lxml')
            try:
                for node_in_child in html_child.find_all("script"):
                    if 'type' in node_in_child.attrs and node_in_child.attrs['type'] == 'application/ld+json':
                        for script_ in node_in_child.contents:
                            jsonStr = script_.strip()
                            jsonList = json.loads(jsonStr) #list
                            if type(jsonList) == list:
                                article_dict = {}
                                for dict_item in jsonList:
                                    if 'articleBody' in dict_item.keys():
                                        article_dict['title']= address[-1].split('.html')
                                        article_dict['source']= url
                                        article_dict['genre']= 'news-article-' + genre
                                        article_dict['content']=dict_item['articleBody']
                                        article_dict['level']='B2'
                                        el_pais_articles.append(article_dict)
                                        count += 1
                                        
                            elif type(jsonList) == dict and 'articleBody' in jsonList.keys():
                                article_dict = {}
                                article_dict['title']=address[-1].split('.html')[0]
                                article_dict['source']=url
                                article_dict['genre']='news-article-' + genre
                                article_dict['content']=jsonList['articleBody']
                                article_dict['level']='B2'
                                el_pais_articles.append(article_dict)
                                count +=1
            except:
                pass
            f_child.close()
            #break
            if count % 100 == 0 and count > 0:
                print(f'scraped {count} articles')
            if count % 1000 == 0 and count > 0:
                time.sleep(1)
            if count == max_num_articles:
                break
    f.close()
    return el_pais_articles

In [4]:
el_pais_articles = scraping_el_pais(url)

In [5]:
today = date.today()
today_str = today.strftime("%b-%d-%Y")

## Store the data in el_pais.json file for the articles

In [6]:
filename = '../corpus/newspapers/el_pais-'+today_str+'.json'
with open(filename, 'w', encoding = 'utf-8') as outfile:
        json.dump(el_pais_articles, outfile)

## Read the el_pais.json file 

In [34]:
with open(filename, encoding = 'utf-8') as json_file:
    data = json.load(json_file)

In [35]:
print(data[0].keys())
print(len(data[0]['content']))

dict_keys(['title', 'source', 'genre', 'content', 'level'])
6197


In [37]:
print(f'title: {data[0]["title"]}\n')
print(f'source: {data[0]["source"]}\n')
print(f'genre: {data[0]["genre"]}\n')
print(f'content: {data[0]["content"]}\n')
print(f'level: {data[0]["level"]}')

title: la-encuesta-a-pie-de-urna-confirma-la-tendencia-al-alza-de-fujimori-en-las-elecciones-de-peru

source: https://elpais.com/america/

genre: news-article-internacional

content: Las elecciones de Perú no tendrán un ganador claro hasta el último minuto. La igualdad es máxima. La victoria se fraguará voto a voto. Con más del 90% del voto escrutado, Keiko Fujimori mantiene un 50,1% de los apoyos frente al 49,9% para Pedro Castillo, dos décimas de una distancia que se acorta conforme avanza el conteo. El organismo electoral había advertido, con el 40% de las papeletas escrutadas, de que “el voto rural y el voto de selva” es el último en reflejarse en los resultados. Esos son feudos favorables al maestro de escuela. En el resultado tampoco está reflejado aún el voto exterior, que acumula un enorme retraso, con apenas el 11% del conteo. Casi 750.000 peruanos estuvieron habilitados para votar desde el exterior, un voto que, por ahora, favorece a la líder conservadora. Con los dos aspiran

In [160]:
#print(html_elpais.prettify())
################ This piece of code is for debugging purpose only - please ignore. 

for node in html_elpais.find_all("a"):
    if node.attrs['href'].startswith('/') and node.attrs['href'].endswith('.html'):
        address = node.attrs['href'].split('/')
        print(address)
        child_url = 'https://elpais.com' + node.attrs['href']
        f_child = urlopen(child_url)
        html_child = BeautifulSoup(f_child, 'lxml')
        for node_in_child in html_child.find_all("script"):
            if 'type' in node_in_child.attrs and node_in_child.attrs['type'] == 'application/ld+json':
                for script_ in node_in_child.contents:
                    jsonStr = script_.strip()
                    jsonList = json.loads(jsonStr) #list
                    if type(jsonList) == list:
                        for dict_item in jsonList:
                            if 'articleBody' in dict_item.keys():
                                print(dict_item['articleBody'], '\n')
                    elif type(jsonList) == dict and 'articleBody' in jsonList.keys():
                        print(jsonList['articleBody'], '\n')
        f_child.close()
        break

['', 'mexico', 'elecciones-mexicanas', '2021-06-04', 'la-ley-del-plomo-tine-de-sangre-las-urnas-de-mexico.html']
A sus rivales les habían pintado bigote, perilla y algunos cuernos. A él, un lazo negro y un mensaje: descanse en paz. Una frase que en otro país, en plena campaña electoral, puede resultar una broma macabra, pero en México estos días significa una sentencia de muerte real. Tres semanas después de que intentaran asesinarlo a balazos, el candidato del PRI a la alcaldía de Morelia (Michoacán), Guillermo Valencia, se mueve con un chaleco antibalas, en una camioneta blindada y rodeado por cuatro hombres armados hasta los dientes. Cuando en la calle un vecino se acerca a saludarlo bruscamente, siente que se le sube el estómago a la garganta, sus músculos se contraen y él solo puede sonreír. La ley del plomo se ha impuesto en una contienda electoral en la que las reglas democráticas más básicas han saltado por los aires. El proceso electoral se ha convertido en el más violento de 